# Imports


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
from collections import deque
import random
import numpy as np

# Environment Description


In [9]:
# This class is a representation of the puddle world and its properties.

class PuddleEnv:
    
    def __init__(self,goal_letter):

      #The reward matrix is a 12x12 grid which holds rewards for each state
      self.reward_matrix = np.zeros((12,12))
      self.fillRewardMatrix()

      #The state state is assigned randomly from four options
      self.startState = np.zeros((1,2))
      self.assignStartState()
      
      #Different puddle worlds ('A','B' or 'C') have different goals
      self.goal = np.array([0,0])
      self.setGoal(goal_letter)
      self.generalActions = np.array([[-1,0],[1,0],[0,-1],[0,1]])

    def assignStartState(self):
      '''
      This function initiates the initial state one of the start states of the
      Start states mentioned in the graph with equal probability
      '''

      random_number = random.random()
      self.startState = np.zeros((1,2))

      if random_number < 1/4:
        self.startState = np.array([5,0])
      elif random_number < 2/4:
        self.startState = np.array([6,0])
      elif random_number < 3/4:
        self.startState = np.array([10,0])
      else:
        self.startState = np.array([11,0])


    def returnReward(self,state):
      '''
      Returns the reward for a state reached in the puddle world
      '''
      return self.reward_matrix[state[0],state[1]]

    def fillRewardMatrix(self):
      '''
      This function populates the reward matrix with values according 
      to the puddle world problem statement
      '''

      # Reward -1 states
      for i in range(3,9):
        self.reward_matrix[2,i] = -1;

      for i in range(3,7):
        self.reward_matrix[i,3] = -1;
        self.reward_matrix[i,8] = -1;

      for i in range(3,8):
        self.reward_matrix[8,i] = -1;

      self.reward_matrix[6,7] = -1;
      self.reward_matrix[7,7] = -1;
      self.reward_matrix[7,3] = -1;

      # Reward -2 states
      for i in range(4,8):
        self.reward_matrix[3,i] = -2;

      for i in range(4,6):
        self.reward_matrix[i,4] = -2;
        self.reward_matrix[i,7] = -2;

      for i in range(4,7):
        self.reward_matrix[7,i] = -2;
      
      self.reward_matrix[5,6] = -2;
      self.reward_matrix[6,4] = -2;
      self.reward_matrix[6,6] = -2;

      # Reward -3 states
      self.reward_matrix[4,5] = -3;
      self.reward_matrix[4,6] = -3;
      self.reward_matrix[5,5] = -3;
      self.reward_matrix[6,5] = -3;

    def setGoal(self,goal_character):
      '''
      This function assigns the goal state depending on the 
      vairant of the puddle problem chosen
      '''

      if goal_character == 'A':
        self.reward_matrix[0,11] = 10
        self.goal = np.array([0,11])
      if goal_character == 'B':
        self.reward_matrix[2,9] = 10
        self.goal = np.array([2,9])
      if goal_character == 'C':
        self.reward_matrix[6,7] = 10
        self.goal = np.array([6,7])

    def nextState(self,state,action):
      '''
      For a given state and action, this function takes into consideration
      the stochasticity of the world and calculates a final state
      '''

      #possible_actions = [up:indx=0,  down:indx=1,  left:indx=2,  right:indx=3]
      possible_actions = np.array([[-1,0],[1,0],[0,-1],[0,1]])

      #Updating possible actions depending on the current state
      #This specifically tackles the states which are in the edes of the map
      if state[1] == 11:
        possible_actions = np.delete(possible_actions,3,0)

      if state[1] == 0:
        possible_actions = np.delete(possible_actions,2,0)
      
      if state[0] == 11:
        possible_actions = np.delete(possible_actions,1,0)
      
      if state[0] == 0:
        possible_actions = np.delete(possible_actions,0,0)


      #If the action cannot be taken at a particular state,
      #We ignore that action and give possible action equal probabilities
      flag = 0

      for i in range(len(possible_actions)):
        comparison = possible_actions[i] == action
        if comparison.all():
          action_index = i
          flag = 1
      

      remaining_actions = np.delete(possible_actions,i,0)
      newState = np.array([0,0])

      if flag == 0:
        random_number = random.random()
        for i in range(len(remaining_actions)):
          if random_number <= (i+1)/len(remaining_actions) and random_number >= i/len(remaining_actions):
            newState = state + remaining_actions[i]
  
      #Determining what the new state will be,
      #We shall calculate one effective action based on probabilities
      else:
        random_number = random.random()
        if random_number <= 0.9:
          newState = state + action

        else:
          for i in range(len(remaining_actions)):
            if random_number <= 0.9 + (i+1)*0.1/len(remaining_actions) and random_number >= 0.9 + i*0.1/len(remaining_actions):
              newState = state + remaining_actions[i]

      
      #Accounting for the westerly wind, with a probability of 0.5
      #The new state might slide to the east by one square

      random_number = random.random()

      if random_number >= 0.5:
        newState[1] += 1
        if newState[1] == 12:
          newState[1] = 11

      return newState

    def calculateTransition(self,state,action):
      '''
      This returns a transition vector which is of the form
      [state,action,next_state,reward,done]
      At a given state, and taking a certain action,
      it adds the next state, reward as well as a boolean if the
      final state has reached the goal
      '''
      newState = self.nextState(state,action)
      reward_temp = self.returnReward(newState)
      tempBool = False

      comparison = newState == self.goal
      if comparison.all():
        tempBool = True
      transition = [[state],[action],[newState],[reward_temp],[tempBool]]
      return transition


In [6]:
class DQNAgent:
    def create_model(self):
        '''
        The neural network we are using here is a three hidden layer 
        network with 32,64 and 16 hidden nodes in each of these layers.
        It has an output layer with 4 linear 
        '''
        model = Sequential()
        model.add(Dense(32, input_shape=(1,2)))
        model.add(Dense(64))
        model.add(Dense(16))
        model.add(Dense(4, activation='linear')) 
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=['accuracy'])
        return model

    def __init__(self):

        # Main model
        self.model = self.create_model()

        # Target network
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        # An array with last n steps for training
        self.replay_memory = deque(maxlen=10_000)

        # An array with actions that the agent can take
        self.generalActions = np.array([[-1,0],[1,0],[0,-1],[0,1]])

        

    # Adds step's data to a memory replay array
    # (observation space, action, reward, new observation space, done)
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)

    def get_q(self, state):
        return self.model.predict(np.array(state).reshape(-1,1,2))

    def model_next_action(self,state):
      tempQValues = dqnagent.get_q(state.reshape(1,1,2))
      bestActionIndex = np.argmax(tempQValues.flatten())
      action = np.array(self.generalActions[bestActionIndex])
      return action 

    def train_model(self):

      if len(self.replay_memory) < 32:
            return

      minibatch = random.sample(dqnagent.replay_memory, 32)

      current_states = np.array([np.array(transition[0]).flatten() for transition in minibatch])
      current_qs_list = dqnagent.model.predict(current_states.reshape(len(minibatch),1,2))
     
      new_current_states = np.array([np.array(transition[2]).flatten() for transition in minibatch])
      future_qs_list = self.target_model.predict(new_current_states.reshape(len(minibatch),1,2))

      X = []
      y = []

      for index, (current_state, action, new_current_state,reward, done) in enumerate(minibatch):
        if not [done]:
          max_future_q = np.max(future_qs_list[index])
          new_q = reward + DISCOUNT * max_future_q
        else:
          new_q = reward

  # Update Q value for given state
        current_qs = np.array(current_qs_list[index]).flatten()
        for k in range(4):
          comparison = puddle.generalActions[k] == action
          if comparison.all():
            action_index = k
        current_qs[action_index] = new_q[0]

              # And append to our training data
        X.append(current_state)
        y.append(current_qs)
      
      dqnagent.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False)



# Training the model

In [ ]:
#Some initial values are declared here for the ease of changing


epsilon = 1
epsilon_decay = 0.9975
epsilon_min = 0.0001

#An agent is created
dqnagent = DQNAgent()

#These are a few metrics to measure/track the performance at each episode
steps = 0
total_episodes = 0
no_of_episodes_before_targetnetwork_update = 0

#Here, we specifically solve for the 'A' variant of the puddle
#A simple change in goal here adjusts the code to the user's choice
puddle = PuddleEnv('A')
puddle.assignStartState()
temp_state = puddle.startState




for episodes in range(5000):
  #One episode is explained here below
  print("epsilon",epsilon)

  puddle.assignStartState()
  temp_state = puddle.startState
  
  steps = 0
  end_bool = False
  print("episode number",episodes)

  while end_bool == False and steps <= 100:
    #Coding down the epsilon greedy concept

    if random.random() < epsilon :
      #Condition below accounts for exploration
      temp_action = np.array(puddle.generalActions[random.randint(0,3)])
      temp_transition = puddle.calculateTransition(temp_state,temp_action)
      dqnagent.update_replay_memory(temp_transition)

      steps += 1
      temp_state = np.array(temp_transition[2]).flatten()
      end_bool = np.array(temp_transition[4])[0]

    else:
      #Condition below accounts for exploitation
      tempQValues = dqnagent.get_q(temp_state.reshape(1,1,2))
      bestActionIndex = np.argmax(tempQValues.flatten())
      temp_action = np.array(puddle.generalActions[bestActionIndex])
      temp_transition = puddle.calculateTransition(temp_state,temp_action) 
      dqnagent.update_replay_memory(temp_transition)

      steps += 1
      temp_state = np.array(temp_transition[2]).flatten()
      end_bool = np.array(temp_transition[4])[0]

  if epsilon > epsilon_min:
    epsilon *= epsilon_decay
    epsilon = max(epsilon,epsilon_min)

  print("steps",steps)
  

  print("memorysize",len(dqnagent.replay_memory))

  

  #Train DQN now
  '''
  At this point of the code, we collect mini batches of size 32 from replay 
  memory, use the target network to update targets for the main network to 
  learn from and train the main network.
  A total of 10 batches are taken for each episode
  '''
  for batches in range(10):

    #if replay memory is too small, we dont train and continue with more 
    #episodes for more data

    if len(dqnagent.replay_memory) < 32:
      break
    # A mini batch of size 32 is taken 
    minibatch = random.sample(dqnagent.replay_memory, 32)

    current_states = np.array([np.array(transition[0]).flatten() for transition in minibatch])
    current_qs_list = dqnagent.model.predict(current_states.reshape(len(minibatch),1,2))
    new_current_states = np.array([np.array(transition[2]).flatten() for transition in minibatch])
    future_qs_list = dqnagent.target_model.predict(new_current_states.reshape(len(minibatch),1,2))

    # empty arrays to populate for the network to learn from
    X = []
    y = []

    # Calculating the maximum q for the action from the target network
    for index, (current_state, action, new_current_state,reward, done) in enumerate(minibatch):
      if not [done]:
        max_future_q = np.max(future_qs_list[index])
        new_q = reward + DISCOUNT * max_future_q
      else:
        new_q = reward

    # Update Q value for given state
      current_qs = np.array(current_qs_list[index]).flatten()
      for k in range(4):
        comparison = puddle.generalActions[k] == action
        if comparison.all():
          action_index = k
      current_qs[action_index] = new_q[0]

     # And append to our training data
      X.append(current_state)
      y.append(current_qs)

    #Finally the 'model' of the agent is trained
    dqnagent.model.fit(np.array(X), np.array(y), batch_size=32, verbose=0, shuffle=False)

  no_of_episodes_before_targetnetwork_update += 1
  total_episodes += 1

  # Check if '5' episodes are done so we can update our target network as well
  if no_of_episodes_before_targetnetwork_update >= 5:
    dqnagent.target_model.set_weights(dqnagent.model.get_weights())
    no_of_episodes_before_targetnetwork_update = 0
    print("Network target updated")


In [ ]:
'''
This snippet of the code, starts with the start state = [5,0]
predicts the best action to take using the model and counts the steps it takes 
to reach the goal state 'A' = [0,11]
'''

sum = 0
for i in range(20):
  boolCompleted = [False]
  stepCount = 0
  tempState = np.array([5,0])
  print('initial state',tempState)

  while(boolCompleted == [False]):
    tempQValues = dqnagent.get_q(tempState.reshape(1,1,2))
    bestActionIndex = np.argmax(tempQValues.flatten())
    tempAction = np.array(puddle.generalActions[bestActionIndex])
    tempTransition = puddle.calculateTransition(tempState,tempAction) 
    
    boolCompleted = tempTransition[4]
    stepCount += 1
    tempState = np.array(tempTransition[2]).flatten()

  print(stepCount)
  print('---------------------')
  sum += stepCount


print('===========================================')
print('Average of 20 runs',sum/20)





In [ ]:
'''
This snippet of the code, starts with the start state = [6,0]
predicts the best action to take using the model and counts the steps it takes 
to reach the goal state 'A' = [0,11]
'''

sum = 0
for i in range(20):
  boolCompleted = [False]
  stepCount = 0
  tempState = np.array([6,0])
  print('initial state',tempState)

  while(boolCompleted == [False]):
    tempQValues = dqnagent.get_q(tempState.reshape(1,1,2))
    bestActionIndex = np.argmax(tempQValues.flatten())
    tempAction = np.array(puddle.generalActions[bestActionIndex])
    tempTransition = puddle.calculateTransition(tempState,tempAction) 
    
    boolCompleted = tempTransition[4]
    stepCount += 1
    tempState = np.array(tempTransition[2]).flatten()

  print(stepCount)
  print('---------------------')
  sum += stepCount


print('===========================================')
print('Average of 20 runs',sum/20)


In [ ]:
'''
This snippet of the code, starts with the start state = [10,0]
predicts the best action to take using the model and counts the steps it takes 
to reach the goal state 'A' = [0,11]
'''

sum = 0
for i in range(20):
  boolCompleted = [False]
  stepCount = 0
  tempState = np.array([10,0])
  print('initial state',tempState)

  while(boolCompleted == [False]):
    tempQValues = dqnagent.get_q(tempState.reshape(1,1,2))
    bestActionIndex = np.argmax(tempQValues.flatten())
    tempAction = np.array(puddle.generalActions[bestActionIndex])
    tempTransition = puddle.calculateTransition(tempState,tempAction) 
    
    boolCompleted = tempTransition[4]
    stepCount += 1
    tempState = np.array(tempTransition[2]).flatten()

  print(stepCount)
  print('---------------------')
  sum += stepCount


print('===========================================')
print('Average of 20 runs',sum/20)


In [ ]:
'''
This snippet of the code, starts with the start state = [11,0]
predicts the best action to take using the model and counts the steps it takes 
to reach the goal state 'A' = [0,11]
'''


sum = 0
for i in range(20):
  boolCompleted = [False]
  stepCount = 0
  tempState = np.array([11,0])
  print('initial state',tempState)

  while(boolCompleted == [False]):
    tempQValues = dqnagent.get_q(tempState.reshape(1,1,2))
    bestActionIndex = np.argmax(tempQValues.flatten())
    tempAction = np.array(puddle.generalActions[bestActionIndex])
    tempTransition = puddle.calculateTransition(tempState,tempAction) 
    
    boolCompleted = tempTransition[4]
    stepCount += 1
    tempState = np.array(tempTransition[2]).flatten()

  print(stepCount)
  print('---------------------')
  sum += stepCount


print('===========================================')
print('Average of 20 runs',sum/20)
